# Comparison of model-based and MRI-based target volume definition for ocular proton therapy 
These scripts provide methods to generate polynomial tumour models, as commonly used in ocular proton therapy planning, based on 3D tumour delineations. Furthermore, it provides methods to compare the resulting model to the tumour delineations. 

More figures and details about the methods are available within the functions.

L. Klaassen, 2025. 


In [1]:
from Prepare_base import correct_base, redefine_prom, expand_base
from Generate_tumour_model import generate_tumour_model, generate_tumour_model_extrathickness, upsample_point_cloud, save_point_cloud_as_stl, directed_angle_between_vectors
from Model_evaluation import volume_analysis, signed_surface_dist
from Automatic_measurements import Prom_Centre, LBD

import trimesh
import os
import numpy as np
import pandas as pd
import alphashape
import pymeshfix
import scipy
import warnings
import shapely
import matplotlib.pyplot as plt
import sklearn
import manifold3d
from tqdm.auto import tqdm 

### Preparations

In [2]:
# Define paths
filedir = r'' #Enter the path where STL files are
outdir = r'' # Enter the path where you want the models to go

tumour_path = os.path.join(filedir, 'Tumour_example.stl') 
eye_path = os.path.join(filedir, 'Eye_example.stl')

if not os.path.isdir(outdir):
    os.makedirs(outdir)


In [3]:
#Load stl 
tumour = trimesh.load(tumour_path)
eye = trimesh.load(eye_path)

# Tumour measurements
thickness, thickness_base, thickness_top = Prom_Centre(tumour,eye, include_sclera = False) # Prominence = thickness
lbd, lbd_coor1, lbd_coor2, base = LBD(tumour,eye)
print('Thickness is ', round(thickness,1), 'mm, largest diameter is ', round(lbd,1), 'mm')

# Define tumour base and expanded tumour base
corrected_base, corrected_base_normals = correct_base(tumour,eye, threshold_angle = 45)  
expanded_base, new_points = expand_base(corrected_base, max_distance = 1.0) 

# Upsample point clouds for tumour base
corrected_base, new_points2 = upsample_point_cloud(corrected_base, factor=10, k=10) # Upsample base by factor 10 with 10 nearest neighbours

# Check if thickness needs to be corrected (automatic thickness determination is less accurate in tumours with middle of the eye within or very close to tumour)
thickness_base, redefined = redefine_prom(tumour,eye, corrected_base, thickness_base)
        

Thickness is  9.1 mm, largest diameter is  17.7 mm


### Construct tumour models

In [4]:
# Construct tumour models: standard version without expansions
for sf in tqdm(np.arange(1,10,1)): #make models for shape factors 1-10 in steps of 1 
    try:
        sf = round(sf,1)
            
        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_sf{0}.stl'.format(sf)) 

        tumour_model_points = generate_tumour_model(tumour,eye, corrected_base, sf) #generate tumour model
        save_point_cloud_as_stl(tumour_model_points, out, max_attempts=5) #save tumour model as STL
                        
        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points[:,0], tumour_model_points[:,1], tumour_model_points[:,2], c= 'b', alpha = 0.7, label = 'tumour model')
    
        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)
                


  0%|          | 0/9 [00:00<?, ?it/s]

   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_sf1.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_sf2.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_sf3.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_sf4.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_sf5.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Mesh not a volume, upsampling points (Attempt 3)...
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_sf6.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model

In [5]:
# Construct tumour models: version with extra thickness 
for sf in tqdm(np.arange(1,10,1)): #Make models for shape factors 1-10 in steps of 1 
    try:
        sf = round(sf,1)
            
        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_extrathickness_sf{0}.stl'.format(sf)) 
                    
        tumour = trimesh.load(tumour_path)
        eye = trimesh.load(eye_path)

        
        tumour_model_points_extrathickness = generate_tumour_model_extrathickness(tumour,eye, corrected_base, sf, addedthickness = 0.5) #generate tumour model with 0.5 mm added thickness
        save_point_cloud_as_stl(tumour_model_points_extrathickness, out, max_attempts=5) #save tumour model as STL
                        
        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points_extrathickness[:,0], tumour_model_points_extrathickness[:,1], tumour_model_points_extrathickness[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)
                

  0%|          | 0/9 [00:00<?, ?it/s]

   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrathickness_sf1.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrathickness_sf2.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrathickness_sf3.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrathickness_sf4.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrathickness_sf5.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testp

In [6]:
# Construct tumour models: version with expanded base
for sf in tqdm(np.arange(1,10,1)): #make models for shape factors 1-10 in steps of 1 
    try:
        sf = round(sf,1)
            
        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_extrabase_sf{0}.stl'.format(sf))
                    
        tumour = trimesh.load(tumour_path)
        eye = trimesh.load(eye_path)
   
        tumour_model_points_extrabase = generate_tumour_model(tumour,eye, expanded_base, sf) #generate tumour model 
        save_point_cloud_as_stl(tumour_model_points_extrabase, out, max_attempts=5) #save tumour model as STL
                        
        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points_extrabase[:,0], tumour_model_points_extrabase[:,1], tumour_model_points_extrabase[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)


  0%|          | 0/9 [00:00<?, ?it/s]

   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrabase_sf1.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrabase_sf2.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrabase_sf3.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrabase_sf4.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrabase_sf5.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrabase_sf6.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrabase_sf7.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (p

In [7]:
# Construct tumour models: version with added thickness and expanded base
for sf in tqdm(np.arange(1,10,1)): #make models for shape factors 1-10 in steps of 1 
    try:
        sf = round(sf,1)
            
        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_extrathickness_extrabase_sf{0}.stl'.format(sf))
                    
        tumour = trimesh.load(tumour_path)
        eye = trimesh.load(eye_path)

        tumour_model_points_extrathickness_extrabase = generate_tumour_model_extrathickness(tumour,eye, expanded_base, sf, addedthickness = 0.5) #generate tumour model with 0.5 mm added thickness
        save_point_cloud_as_stl(tumour_model_points_extrathickness_extrabase, out, max_attempts=5) #save tumour model as STL
                        
        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points_extrathickness_extrabase[:,0], tumour_model_points_extrathickness_extrabase[:,1], tumour_model_points_extrathickness_extrabase[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)

  0%|          | 0/9 [00:00<?, ?it/s]

   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrathickness_extrabase_sf1.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrathickness_extrabase_sf2.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrathickness_extrabase_sf3.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrathickness_extrabase_sf4.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrathickness_extrabase_sf5.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extrathickness_extrabase_sf6.stl
   Saved STL file: U:\LisaKlaassen\Jupyter\Model_based_MR_based\To GitHub (public)\Models\Testpatient_tumourmodel_extr

### Evaluate tumour models 
If tumour models have unexpected shapes, sometimes it is sufficient to run the model generation again for that shape factor.

In [8]:
# Evaluation for standard model 
volume_metrics_perpatient = {}
distance_metrics_perpatient = {}
        
for sf in tqdm(np.arange(1,10,1)):
    try:
        sf = round(sf,1)
        tqdm.write('   started with sf {0}'.format(sf))
        model_path = os.path.join(outdir, 'Testpatient_tumourmodel_sf{0}.stl'.format(sf)) 
        model = trimesh.load(model_path)
        eye = trimesh.load(eye_path)
                
        volume_metrics = volume_analysis(tumour, model)
        volume_metrics_perpatient[r'SF{0}'.format(sf)] = volume_metrics

        dists, dist_metrics = signed_surface_dist(tumour, model, eye)
        distance_metrics_perpatient[r'SF{0}'.format(sf)] = dist_metrics
        #print('    ', dist_metrics)
    except Exception as e:
        print(e)
        volume_metrics_perpatient[r'SF{0}'.format(sf)] = {'overlap_abs': np.nan, 'overlap_rel': np.nan, 'overestimation_abs': np.nan,
                'overestimation_rel': np.nan, 'underestimation_abs': np.nan, 
                'underestimation_rel': np.nan, 'IoU': np.nan} 
        distance_metrics_perpatient[r'SF{0}'.format(sf)] = {'surf_dist_median_abs':  np.nan, 'surf_dist_min': np.nan, 'surf_dist_perc_0.5': np.nan,'surf_dist_perc_1': np.nan,'surf_dist_perc_2': np.nan,
                                                            'surf_dist_perc_5':  np.nan, 'surf_dist_perc_25':  np.nan, 
                'surf_dist_perc_50':  np.nan, 'surf_dist_perc_75':  np.nan, 'surf_dist_perc_95':  np.nan, 'surf_dist_max': np.nan}


  0%|          | 0/9 [00:00<?, ?it/s]

   started with sf 1
   started with sf 2
   started with sf 3
   started with sf 4
   started with sf 5
   started with sf 6
   started with sf 7
   started with sf 8
   started with sf 9


In [9]:
# Put the results in a dataframe
rows = []
rows2 = []

for sf, metrics in volume_metrics_perpatient.items():
    sf_number = float(sf.replace('SF', ''))
    rows.append({
            'SF': sf_number,
            'overlap_abs': metrics['overlap_abs'],
            'overlap_rel': metrics['overlap_rel'],
            'overestimation_abs': metrics['overestimation_abs'],
            'overestimation_rel': metrics['overestimation_rel'],
            'underestimation_abs': metrics['underestimation_abs'],
            'underestimation_rel': metrics['underestimation_rel'],
            'IoU': metrics['IoU'],
        })


for sf, metrics in distance_metrics_perpatient.items():
    sf_number = float(sf.replace('SF', ''))
    rows2.append({
            'SF': sf_number,
            'surf_dist_median_abs': metrics['surf_dist_median_abs'],
            'surf_dist_min': metrics['surf_dist_min'],
            'surf_dist_perc_0.5': metrics['surf_dist_perc_0.5'],
            'surf_dist_perc_1': metrics['surf_dist_perc_1'],
            'surf_dist_perc_2': metrics['surf_dist_perc_2'],
            'surf_dist_perc_5': metrics['surf_dist_perc_5'],
            'surf_dist_perc_25': metrics['surf_dist_perc_25'],
            'surf_dist_perc_50': metrics['surf_dist_perc_50'],
            'surf_dist_perc_75': metrics['surf_dist_perc_75'],
            'surf_dist_perc_95': metrics['surf_dist_perc_95']
        })

volume_df = pd.DataFrame(rows)
distance_df = pd.DataFrame(rows2)

allmetrics_df = pd.DataFrame.merge(volume_df, distance_df, how = 'left')


In [10]:
# Export dataframe
out_excel_standardmodel = os.path.join(outdir, 'Evaluation_standardmodel.xlsx')
allmetrics_df.to_excel(out_excel_standardmodel)
allmetrics_df

,SF,overlap_abs,overlap_rel,overestimation_abs,overestimation_rel,underestimation_abs,underestimation_rel,IoU,surf_dist_median_abs,surf_dist_min,surf_dist_perc_0.5,surf_dist_perc_1,surf_dist_perc_2,surf_dist_perc_5,surf_dist_perc_25,surf_dist_perc_50,surf_dist_perc_75,surf_dist_perc_95
0,1.0,715.234285,65.939292,0.421971,0.038903,369.451737,34.060708,0.659137,1.401788,-2.343990,-2.280602,-2.237242,-2.179070,-2.078832,-1.471152,-0.635873,1.324371,2.027386
1,2.0,1041.785978,96.044934,27.103592,2.498750,42.900038,3.955065,0.937035,0.253767,-0.723930,-0.641024,-0.595674,-0.502933,-0.378530,-0.133411,0.177483,0.327760,0.496815
2,3.0,1070.633489,98.704461,186.917962,17.232449,14.052535,1.295539,0.841955,0.667472,-0.502916,-0.403575,-0.331821,-0.237543,0.096539,0.356064,0.667472,0.944096,1.318359
3,4.0,1072.319088,98.859860,306.118599,28.221863,12.366919,1.140138,0.771006,1.052425,-0.389095,-0.295261,-0.256912,-0.134268,0.156217,0.544952,1.052425,1.466336,1.913746
4,5.0,1074.055027,99.019901,388.613395,35.827270,10.630998,0.980099,0.729013,1.241519,-0.381083,-0.225419,-0.170789,0.061383,0.177528,0.635607,1.241519,1.801598,2.302267
5,6.0,1073.529243,98.971428,446.922634,41.202949,11.156785,1.028573,0.700916,1.357725,-0.346130,-0.190054,-0.117785,0.093541,0.181165,0.702570,1.357725,2.017791,2.610367
6,7.0,1073.661233,98.983596,495.196149,45.653409,11.024786,1.016403,0.679583,1.477748,-0.402946,-0.190327,-0.117782,0.107101,0.204028,0.771128,1.477748,2.161458,2.880111
7,8.0,1071.340823,98.769672,530.065452,48.868100,13.345206,1.230329,0.663471,1.502730,-0.345359,-0.175170,-0.110113,0.105179,0.201084,0.790677,1.502730,2.253044,3.114341
8,9.0,1076.028336,99.201826,558.257645,51.467211,8.657696,0.798175,0.654939,1.574215,-0.372612,-0.170655,0.055724,0.124920,0.216837,0.830723,1.574215,2.347053,3.318774
